In [1]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
import pandas as pd
import matplotlib.pyplot as plt

import g4ppyy as g4
from g4ppyy import mm, cm, m, eV, MeV, GeV, deg, twopi

import random

gRunManager = g4.G4RunManager();

ModuleNotFoundError: No module named 'g4ppyy'

In [ ]:
class custom_generator(g4.G4VUserPrimaryGeneratorAction):
    "Flat Energy Spectrum Upward Neutron Generator"
    def __init__(self):
        super().__init__()
        self.particle = g4.G4Neutron.Definition()
        self.particleGun = g4.G4ParticleGun()
        self.particleGun.SetParticleDefinition(self.particle)
        self.particleGun.SetParticleMomentumDirection(g4.G4ThreeVector(0, 0, +1))
        self.particleGun.SetParticlePosition(g4.G4ThreeVector(0.0,0.0,0.0))
        self.particleGun.SetNumberOfParticles(1)  
        self.particleGun.SetParticleTime(0)

    def GeneratePrimaries(self, anEvent):
        energy = random.random() * 20 * MeV
        self.particleGun.SetParticleEnergy(energy)  
        self.particleGun.GeneratePrimaryVertex(anEvent)

In [ ]:
# In [3]: NEUTRON DETECTOR
# Make a ND array to store neutron energy
class neutron_energy_store(g4.G4VSensitiveDetector):

    def Reset(self):
        self.neutron_event = {
            "eid": [],
            "pid": [],
            "ke": [],
            "x": [],
            "y": [],
            "z": [],
        }
                
    def ProcessHits(self, aStep, ROhist):
        
        pdg = (aStep.GetTrack().GetParticleDefinition().GetPDGEncoding())
        if not (pdg == 2112): return 0

        # Get Before
        eid = int(gRunManager.GetCurrentEvent().GetEventID())
        pid = int(aStep.GetTrack().GetTrackID())
        pos = aStep.GetPreStepPoint().GetPosition() 
        ke = aStep.GetPreStepPoint().GetKineticEnergy() 
        
        self.neutron_event["eid"].append(eid)
        self.neutron_event["pid"].append(pid)
        self.neutron_event["ke"].append(ke)
        self.neutron_event["x"].append(pos.x())
        self.neutron_event["y"].append(pos.y())
        self.neutron_event["z"].append(pos.z())
        
        pos = aStep.GetPostStepPoint().GetPosition() 
        ke = aStep.GetPostStepPoint().GetKineticEnergy() 

        self.neutron_event["eid"].append(eid)
        self.neutron_event["pid"].append(pid)
        self.neutron_event["ke"].append(ke)
        self.neutron_event["x"].append(pos.x())
        self.neutron_event["y"].append(pos.y())
        self.neutron_event["z"].append(pos.z())

        # Comment this out to kill the neutron after it's seen.
        aStep.GetTrack().SetTrackStatus(g4.G4TrackStatus.fStopAndKill)

        return 1

    def StartOfRunAction(self):
        self.Reset()

    def EndOfRunAction(self):
        self.df = pd.DataFrame(data=self.neutron_event)

    def VisualizationAction(self):
        plt.scatter(self.df.x, self.df.z, c=self.df.ke)
        plt.title(str(self.GetName()) + " : X-Y-KE")
        plt.xlabel("x [mm]")
        plt.ylabel("z [mm]")
        plt.colorbar()
        plt.show()
                

In [ ]:
# In[3]: BUILD A WORLD consisting of AIR with a HDPE tube around source.
class custom_world(g4.G4VUserDetectorConstruction):   
    
    def BuildWorld(self):
        # Mother Box
        self.world = g4.build_component("world", solid="box", x=5*m, y=5*m, z=5*m, material="G4_AIR")

        # Moderator thickness
        thickness = 1*cm
        
        # World Geometries
        self.hdpe_outer = g4.build_component("shell", solid="tubs", rot=[90*deg, 0.0, 0.0], rmax=22*cm, z=1.2*m/2, mother=self.world, 
material="G4_POLYETHYLENE", color=[0.0,0.0,1.0,0.8], drawstyle="wireframe")
        self.air_inner = g4.build_component("air", solid="tubs", rmax=22*cm-thickness, z=1.2*m/2, mother=self.hdpe_outer, material="G4_AIR", 
color=[0.5,0.5,1.0,0.1], drawstyle="wireframe")

        self.nai_crystal = g4.build_component("nai", solid="tubs", rmax=20*cm, z=20*cm/2, mother=self.world, material="G4_SODIUM_IODIDE", 
pos=[0.0,0.0,+40*cm])
    
    def BuildDetectors(self):
        # Sensitive Volume Definitions
        self.nai_det = neutron_energy_store("nai_det")
        self.nai_crystal.GetLogicalVolume().SetSensitiveDetector(self.nai_det)
        g4.register_detector_hooks(detector.nai_det)

    def Construct(self):
        self.BuildWorld()
        self.BuildDetectors()
        
        # Return the mother
        return self.world # top mother volume

In [ ]:
# In [4]:
# Add Physics List
physics = g4.QGSP_BERT_HP()
gRunManager.SetUserInitialization(physics)

# Add a World
detector = custom_world()
gRunManager.SetUserInitialization(detector)

# Add a Generator
gen = custom_generator()
gRunManager.SetUserAction(gen)

# Add standard GEANT4 Actions
g4.add_default_actions(gRunManager)

# Setup vis to check geometry (optioonal)
g4.create_visualization(gRunManager)

# Generate some events
g4.handle_beam(gRunManager, 10000)

# Draw the vis plot (optional)
g4.draw_visualization(gRunManager)

print("FINISHED")